Générer des playlists :
1 avec le genre pref de la personne 
1 avec les sorties des artistes pref
1 pour s'ambiancer et bouger
1 pour se concentrer/se calmer
1 avec des morceaux pour découvrir quelque chose qu'on n'écoute pas

In [5]:
import json
from datetime import datetime
import random

# Charger les données depuis les fichiers
with open('../data/caracteristiques_fictives.json', 'r') as musiques_file:
    musiques_data = json.load(musiques_file)

with open('../data/utilisateurs_fictifs.json', 'r') as utilisateurs_file:
    utilisateurs_data = json.load(utilisateurs_file)


# Générer un titre dynamique pour une playlist
def generate_playlist_title(base_title):
    date_str = datetime.now().strftime("%d %B %Y")
    return f"{base_title} - {date_str}"

# Générer une playlist basée sur les préférences d'un utilisateur
def generate_playlist(user, tracks, genre_filter=None, tempo_range=None, exclude_genres=None, liked_tracks_only=False, size=10):
    filtered_tracks = []

    # Filtrer par genre et autres critères
    for track in tracks:
        # Filtrer par genre
        if genre_filter and not any(genre in track['Genres'] for genre in genre_filter):
            continue
        # Exclure certains genres
        if exclude_genres and any(genre in track['Genres'] for genre in exclude_genres):
            continue
        # Filtrer par tempo
        if tempo_range and not (tempo_range[0] <= track['Tempo'] <= tempo_range[1]):
            continue
        # Si on veut seulement des morceaux aimés
        if liked_tracks_only and track['Id'] not in user['preferences']['liked_tracks']:
            continue
        # Ne pas inclure des morceaux favoris dans la 5ème playlist
        if track['Id'] in user['preferences']['liked_tracks']:
            continue
        filtered_tracks.append(track)

    # Retourner une liste d'ID de morceaux au lieu des morceaux complets
    return [track['Id'] for track in random.sample(filtered_tracks, min(size, len(filtered_tracks)))]

# Créer 5 playlists pour chaque utilisateur
user_playlists = {}

for user in utilisateurs_data['users']:
    playlists = {}

    # Playlist 1: "100% [Genre préféré]"
    main_genre = user['preferences']['genres'][0] if user['preferences']['genres'] else "Diverse"
    playlists[generate_playlist_title(f"100% {main_genre}")] = generate_playlist(
        user,
        musiques_data['tracks'],
        genre_filter=[main_genre],
        size=10
    )

    # Playlist 2: "Sorties des artistes préférés" (utilise les morceaux aimés de l'utilisateur)
    playlists[generate_playlist_title(f"Sorties {main_genre}")] = generate_playlist(
        user,
        musiques_data['tracks'],
        genre_filter=[main_genre],
        liked_tracks_only=True,
        size=10
    )

    # Playlist 3: "Pour s'ambiancer et bouger" (musique énergique, tempo rapide)
    playlists[generate_playlist_title("Pour s'ambiancer et bouger")] = generate_playlist(
        user,
        musiques_data['tracks'],
        tempo_range=[120, 160],
        liked_tracks_only=True,
        size=10
    )

    # Playlist 4: "Pour se concentrer/se calmer" (genres instrumentaux, tempo lent)
    playlists[generate_playlist_title("Pour se concentrer/se calmer")] = generate_playlist(
        user,
        musiques_data['tracks'],
        genre_filter=["Classical", "Instrumental"],
        tempo_range=[60, 100],
        liked_tracks_only=True,
        size=10
    )

    # Playlist 5: "Pour découvrir quelque chose qu'on n'écoute pas" (autres genres que ceux de l'utilisateur)
    # Exclure tous les genres préférés
    exclude_genres = user['preferences']['genres']
    all_genres = [track['Genres'] for track in musiques_data['tracks']]
    all_genres_flat = set(genre for sublist in all_genres for genre in sublist)
    remaining_genres = all_genres_flat - set(exclude_genres)

    # Filtrer les morceaux ne correspondant pas aux genres de l'utilisateur et ne pas inclure les morceaux favoris
    playlists[generate_playlist_title("Pour découvrir autre chose")] = generate_playlist(
        user,
        musiques_data['tracks'],
        genre_filter=list(remaining_genres),
        liked_tracks_only=False,
        size=10
    )

    user_playlists[user['id']] = playlists

# Sauvegarder les playlists générées dans un fichier JSON
playlists_file_path = '../data/generated_playlists.json'
with open(playlists_file_path, 'w') as playlists_file:
    json.dump(user_playlists, playlists_file, indent=2)

playlists_file_path

'../data/generated_playlists.json'